<a href="https://colab.research.google.com/github/CherylA24/CreditScoringClassification/blob/main/ClassPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install category_encoders 

     |████████████████████████████████| 80 kB 3.7 MB/s 


In [ ]:
pip install dython



---



In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, roc_curve, precision_score, f1_score, recall_score
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif, SelectKBest
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, MinMaxScaler, QuantileTransformer, LabelEncoder, OneHotEncoder
import category_encoders as ce
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, ComplementNB
import xgboost as xgb
import lightgbm as lgb
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.feature_selection._base import SelectorMixin
from sklearn.utils.sparsefuncs import mean_variance_axis, min_max_axis
from sklearn.utils.validation import check_is_fitted
from sklearn.utils._tags import _safe_tags
from sklearn.utils import (
    check_array,
    safe_mask,
    safe_sqr,
)
# from dython.nominal import associations

import joblib
from os.path import exists as file_exists
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
import itertools

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
class FillCatsNA(BaseEstimator, TransformerMixin):
  def __init__(self):
    pass

  def fit(self, X, y=None):
    return self

  def transform(self, X, y = None):
    X_data = X.copy()

    X_data['jenispinjaman'] = X_data['jenispinjaman'].fillna(value='-')
    X_data['JarakDomisili'] = X_data['JarakDomisili'].fillna(value = 'Kurang atau Sama Dengan 60 KM')

    X_data = X_data.apply(lambda x: x.fillna(x.value_counts().index[0]))
    # X_data = X_data.fillna(value='-')

    return X_data 

In [ ]:
class ConvertData(BaseEstimator, TransformerMixin):
  def __init__(self):
    pass
  
  def fit(self, X, y=None):
    return self
  
  def transform(self, X):
    X_data = X.copy()

    X_data['TotalLamaPernahBekerja'] = X_data['TotalLamaPernahBekerja'].fillna(value = 0)

    def repeated(string):
      for x in range(1, len(string)):
        substring = string[:x]
        if substring * (len(string)//len(substring))+(substring[:len(string)%len(substring)]) == string:
          return substring
        elif (string == '2015'):
          return '215'
        elif (string == '2016'):
          return '216'
        elif (string == '1994'):
          return '228'
        elif (string =='21552263233'):
          return '1'
        else :
          return string[0:3]

      return string

    X_data['TotalLamaPernahBekerja'] = X_data['TotalLamaPernahBekerja'].astype(str)
    X_data['TotalLamaPernahBekerja'] = X_data['TotalLamaPernahBekerja'].str.extract('(\d+)')
    X_data['TotalLamaPernahBekerja'] = X_data['TotalLamaPernahBekerja'].astype(str)
    X_data['TotalLamaPernahBekerja'] = X_data['TotalLamaPernahBekerja'].apply(repeated)
    X_data['TotalLamaPernahBekerja'] = X_data['TotalLamaPernahBekerja'].astype(float)

    return X_data

In [ ]:
class ReverseVarianceThreshold(SelectorMixin, BaseEstimator):

  def __init__(self, threshold=0.0):
    self.threshold = threshold

  def fit(self, X, y=None):

    X = self._validate_data(
        X,
        accept_sparse=("csr", "csc"),
        dtype=np.float64,
        force_all_finite="allow-nan",
    )

    if hasattr(X, "toarray"):  # sparse matrix
      _, self.variances_ = mean_variance_axis(X, axis=0)
      if self.threshold == 0:
        mins, maxes = min_max_axis(X, axis=0)
        peak_to_peaks = maxes - mins
    else:
      self.variances_ = np.nanvar(X, axis=0)
      if self.threshold == 0:
        peak_to_peaks = np.ptp(X, axis=0)

    if self.threshold == 0:
      compare_arr = np.array([self.variances_, peak_to_peaks])
      self.variances_ = np.nanmin(compare_arr, axis=0)
    elif self.threshold < 0.0:
      raise ValueError(f"Threshold must be non-negative. Got: {self.threshold}")

    if np.all(~np.isfinite(self.variances_) | (self.variances_ <= self.threshold)):
      msg = "No feature in X meets the variance threshold {0:.5f}"
      if X.shape[0] == 1:
        msg += " (X contains only one sample)"
      raise ValueError(msg.format(self.threshold))

    return self

  def _get_support_mask(self):
    check_is_fitted(self)

    return self.variances_ < self.threshold

  def transform(self, X):
    X = self._validate_data(
        X,
        dtype=None,
        accept_sparse="csr",
        force_all_finite=not _safe_tags(self, key="allow_nan"),
        reset=False,
    )
    return self._transform(X)

  def _transform(self, X):
    mask = self.get_support()
    if not mask.any():
      warn(
          "No features were selected: either the data is"
          " too noisy or the selection test too strict.",
          UserWarning,
      )
      return np.empty(0).reshape((X.shape[0], 0))
    if len(mask) != X.shape[1]:
      raise ValueError("X has a different shape than during fitting.")
    return X[:, safe_mask(X, mask)]

In [ ]:
class DropCorrelation(BaseEstimator, TransformerMixin):
  def __init__(self, threshold):
    self.threshold = threshold
    self.col_corr = None
  
  def fit(self, X, y = None):
    col_corr = set()
    X = pd.DataFrame(X)
    X['FLAG_BAD'] = y
    corr_matrix = X.corr()
    target_loc = corr_matrix.columns.get_loc("FLAG_BAD")
    for i in range(len(corr_matrix.columns)):
      for j in range(i):
        if abs(corr_matrix.iloc[i, j])> self.threshold:
          if (abs(corr_matrix.iloc[i, target_loc]) < abs(corr_matrix.iloc[j, target_loc])):
            colname = corr_matrix.columns[j]
            col_corr.add(colname)
          else:
            colname = corr_matrix.columns[i]
            col_corr.add(colname)
    
    # col_corr.add('FLAG_BAD')
    self.col_corr = col_corr

    return self
  
  def transform(self, X, y=None, **kwargs):
    return (pd.DataFrame(X)).drop(labels = self.col_corr, axis=1)

In [ ]:
def main():

  c = itertools.count()

  df = pd.read_csv('/content/drive/MyDrive/Data Marginal/MarginalUpd(1).csv')
  df["FLAG_BAD"] = np.where((df["ever30plus"] == 'a. Ever 30Plus') | (df["FlagNegCust"] == '1. NegativeCustomer') , 1, 0)

  cats_cols = ['PersonalType','Gender','MaritalStatus', 'ResidenceCompanyProvince', 'HomeCompanyStatus',
              'Pekerjaan','jabatan','Status_pekerjaan','Jenis_pekerjaan2', 'Pendidikan','HomeFixedLine',
              'KantorFixedLine','Wayofpayment','IsSyariah','ProductType','UsedNew', 'GroupProductId',
              'applicationsource','FirstInstallment','AssetCategory','AssetUsage','MainCoverage', 'GRPNTF',
              'GRPOTR','Bidang_usaha','flagImpact','isoveride', 'ProductOffering', 'GRPDP', 'SektorEkonomi',
              'SupplierType', 'SupplierCategory','InstallmentScheme','AssetType','AssetCategoryId','FlagMerkPopular',
              'MadeIn','isKPM','Jenis_KPM', 'jenispinjaman', 'PaymentMethod','FlagRO','JarakDomisili','FlagNegCust',
              'CompanyCity','ResidenceCompanyCity', 'Nama_Perusahaan','Supplier']
  num_cols = ['ResidenceCompanyZipCode', 'Lama_Menempati_Tahun','Lama_Menempati_Bulan', 'employmentsinceyear',
              'CompanyZipCode', 'NumOfDependence', 'NTFIDR', 'TotalOTR', 'Pendapatan', 
              'DPPercentage', 'CAMOSObligorExposure', 'StartingBalance', 'AvgDebit', 'TotalLamaPernahBekerja',
              'AvgCredit','AvgBalance', 'LastBalanceAmt','GrowthBalance', 'jmltunggakanpokok','jmltunggakanhari']

  target_var=['FLAG_BAD']
  all_var = cats_cols + num_cols + target_var

  df = df[df.columns[df.columns.isin(all_var)]]

  X = df.drop("FLAG_BAD",axis=1)
  y = df["FLAG_BAD"]
    
  X_train, X_test, y_train, y_test= train_test_split(X, y, test_size = 0.3, random_state=0, stratify = y)
  
  def unique_file(basename, ext):
    actualname = "%s.%s" % (basename, ext)
    while file_exists(actualname):
      actualname = "%s (%d).%s" % (basename, next(c), ext)
    return actualname

  def oldFile(basename, ext):
    old_file = "%s (%d).%s" % (basename, next(c)-2 , ext)
    return old_file

  models = {
      'GaussianNB' : GaussianNB(),
      'LogisticRegression' : LogisticRegression(fit_intercept = True, solver = 'saga', max_iter = 5000),
      'RandomForestClassifier' : RandomForestClassifier(n_estimators=100, bootstrap = True, oob_score = True, random_state = 42),
      'XGBClassifier' : xgb.XGBClassifier(objective="binary:logistic", random_state = 42),
      'AdaBoostClassifier' : AdaBoostClassifier(random_state=42),
      'LGBMClassifier' : lgb.LGBMClassifier(random_state = 42)
  }

  model_score = pd.DataFrame(
    columns = ['Model Name','Train Accuracy','Test Accuracy','ROC Value','Precision','Recall','F1 Score'])

  cat_transformer = Pipeline(
    steps=[('imputer', fcna),
          ('encoder2', OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
          ('vt1', rvt),
          ('vt2', VarianceThreshold(threshold = 0.01)),
          ('encoder', ce.WOEEncoder())]
  )

  num_transformer = Pipeline(
    steps = [('convert',cd),
            ('imputer', SimpleImputer(missing_values = np.nan, strategy='constant', fill_value=0.0)),
            ('scaler', QuantileTransformer(output_distribution='normal', n_quantiles = 100000))]
  )

  preprocessor = ColumnTransformer(
    transformers=[('num', num_transformer, num_cols),
                  ('cats', cat_transformer, cats_cols)]
  )

  fitted_models = []
  predictions = []

  file_model_metrics = unique_file("ModelMetrics","txt")

  for key in models:
    pipe = Pipeline(
        steps=[('preprocessor',preprocessor),
              ('dropcorr',dc),
              ('features',SelectKBest(mutual_info_classif, k=20)),
              ('model', models[key])]
    )

    model = pipe.fit(X_train, y_train)
    fitted_models.append(model)

    y_preds = model.predict(X_test)
    predictions.append(y_preds)

    y_probs = model.predict_proba(X_test)[:, 1] 

    train_acc = (1 - (1 - model.score(X_train, y_train))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1)))
    test_acc = (1 - (1 - model.score(X_test, y_test))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))

    with open(file_model_metrics, 'a') as outfile:
      outfile.write("\n")
      outfile.write(key)
      outfile.write("\n====================\n")
      outfile.write("train accuracy: {:.2f}%\n".format(train_acc * 100))
      outfile.write("test accuracy: {:.2f}%\n".format(test_acc * 100))
      outfile.write("roc value : {:.2f}%\n".format((roc_auc_score(y_test, y_probs))*100))
      outfile.write('\nClassification Report\n\n{}\n\nConfusion Matrix\n\n{}\n'.format(classification_report(y_test, y_preds),confusion_matrix(y_test, y_preds)))

    row = {
        'Model Name': key,
        'Train Accuracy' : train_acc,
        'Test Accuracy' : test_acc,
        'ROC Value' : roc_auc_score(y_test,y_probs),
        'Precision' : precision_score(y_test,y_preds),
        'Recall' : recall_score(y_test, y_preds),
        'F1 Score' : f1_score(y_test, y_preds)
      }

    model_score = model_score.append(row, ignore_index = True)

  model_score.set_index("Model Name", inplace = True)
  model_score = model_score.T
  model_score['Best Score'] = model_score.idxmax(axis=1)
  model_score.to_excel(unique_file("ModelScore", "xlsx"))
  # model_score.to_csv(unique_file("ModelScore", "csv"))
  model_score

  if (model_score.loc['ROC Value']['Best Score'] == 'GaussianNB'):
    best_model = fitted_models[0]
    best_roc_value = model_score.loc['ROC Value']['GaussianNB']
    best_preds = predictions[0]

  elif (model_score.loc['ROC Value']['Best Score'] == 'LogisticRegression'):
    best_model = fitted_models[1]
    best_roc_value = model_score.loc['ROC Value']['LogisticRegression']
    best_preds = predictions[1]

  elif (model_score.loc['ROC Value']['Best Score'] == 'RandomForestClassifier'):
    best_model = fitted_models[2]
    best_roc_value = model_score.loc['ROC Value']['RandomForestClassifier']
    best_preds = predictions[2]

  elif (model_score.loc['ROC Value']['Best Score'] == 'XGBClassifier'):
    best_model = fitted_models[3]
    best_roc_value = model_score.loc['ROC Value']['XGBClassifier']
    best_preds = predictions[3]
    
  elif (model_score.loc['ROC Value']['Best Score'] == 'AdaBoostClassifier'):
    best_model = fitted_models[4]
    best_roc_value = model_score.loc['ROC Value']['AdaBoostClassifier']
    best_preds = predictions[4]

  elif (model_score.loc['ROC Value']['Best Score'] == 'LGBMClassifier'):
    best_model = fitted_models[5]
    best_roc_value = model_score.loc['ROC Value']['LGBMClassifier']
    best_preds = predictions[5]  

  def bestModel(params):
    if(best_model == fitted_models[0]):
      clf = GaussianNB(**params)
    elif (best_model == fitted_models[1]):
      clf = LogisticRegression(**params,verbose=0)
    elif (best_model == fitted_models[2]):
      clf = RandomForestClassifier(**params)
    elif (best_model == fitted_models[3]):
      clf = xgb.XGBClassifier(**params)
    elif (best_model == fitted_models[4]):
      clf = AdaBoostClassifier(**params)
    elif (best_model == fitted_models[5]):
      clf = lgb.LGBMClassifier(**params)

    return clf

  def objective (params, n_folds =10, X_train=X_train,y_train=y_train):
    clf = Pipeline(
      steps=[('dropcorr',DropCorrelation(0.9)),
            ('features',SelectKBest(mutual_info_classif, k=20)),
            ('models', bestModel(params))]
    )
    scores = cross_val_score(clf, preprocessor.fit_transform(X_train, y_train), y_train,cv=5,scoring='roc_auc')
    best_scores = max(scores)
    loss = 1-best_scores
    return{'loss':loss, 'params':params, 'status':STATUS_OK}

  if (best_model == fitted_models[0]):
    space={
        'var_smoothing' : hp.uniform('var_smoothing',np.logspace(0, -9, num=100))
    }
  elif (best_model == fitted_models[1]):
    space = {
          'warm_start': hp.choice('warm_start', [True, False]),
          'fit_intercept': hp.choice('fit_intercept', [True, False]),
          'tol': hp.uniform('tol', 0.00001, 0.0001),
          'C': hp.uniform('C', 0.05, 3),
          'solver': hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear','sag']),
          'max_iter': hp.choice('max_iter', range(1000, 5000)),
          'multi_class': 'auto',
          'class_weight': hp.choice('class_weight',[None,'balanced'])
          }
  elif (best_model == fitted_models[2]):
    space = {
          'criterion': hp.choice('criterion', ['entropy', 'gini']),
          'max_depth': hp.quniform('max_depth', 1, 10, 1),
          'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
          'min_samples_leaf': hp.uniform ('min_samples_leaf', 0, 0.5),
          'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
          'n_estimators' : hp.choice('n_estimators', range(10, 1000))
      }
  elif (best_model == fitted_models[3]):
    space = {
          'booster': hp.choice('booster', ['gbtree','gblinear','dart']),
          'max_depth': hp.choice('max_depth', np.arange(1, 10, dtype=int)),
          'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
          'gamma': hp.uniform('gamma', 0, 1),
          'reg_lambda': hp.uniform('reg_lambda', 0, 1),
          'scale_pos_weight': hp.quniform('scale_pos_weight', 1,8,1),
          'subsample' : hp.uniform('subsample',0.3,0.8),
          'colsample_bytree' : hp.uniform('colsample_bytree',0.3,0.8),
          'class_weight': hp.choice('class_weight',[None,'balanced'])
      }
  elif (best_model == fitted_models[4]):
    space = {
          'n_estimators': hp.choice('n_estimators', range(500, 1000)),
          'learning_rate' : hp.uniform('learning_rate', 0.01, 1)
      }
  elif (best_model == fitted_models[5]):
    space = {
          'is_unbalance': hp.choice('is_unbalance', [True, False]),
          'max_depth': hp.choice('max_depth', np.arange(3, 10, dtype=int)),
          'boosting_type': hp.choice('boosting_type', ['gbdt','dart','goss']),
          'num_leaves': hp.choice('num_leaves', np.arange(30, 150, dtype=int)),
          'learning_rate': hp.uniform('learning_rate', 0.0001, 1),
          'max_bin': hp.choice('max_bin', np.arange(50, 500, dtype=int)),
          'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
          'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
          'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
      }
  
  tpe_algor = tpe.suggest
  trials = Trials()
  best = fmin(fn=objective, space = space,algo=tpe.suggest,max_evals=10, trials = trials)
  trials_dict = sorted(trials.results, key = lambda x: x['loss']) 
  hyp_df = pd.DataFrame(trials_dict)
  best_hp = hyp_df.loc[0,'params']
  hp_model = bestModel(best_hp)

  pipe2 = Pipeline(
    steps=[('preprocessor',preprocessor),
          ('dropcorr',DropCorrelation(0.9)),
          ('features',SelectKBest(mutual_info_classif, k=20)),
          ('model', hp_model)]
  )

  hyper_model = pipe2.fit(X_train, y_train)

  hp_preds = hyper_model.predict(X_test)
  hp_probs = hyper_model.predict_proba(X_test)[:, 1]

  hp_roc_value = roc_auc_score(y_test, hp_probs)

  if (best_roc_value > hp_roc_value):
    fin_model = best_model
    fin_preds = best_preds
    fin_roc_value = best_roc_value
  else :
    fin_model = hyper_model
    fin_preds = hp_preds
    fin_roc_value = hp_roc_value

  def savetotxt(filename ,model, preds):
    with open(filename, 'w') as outfile:
      outfile.write("train accuracy: {:.2f}%\n".format((1 - (1 - model.score(X_train, y_train))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1)))*100))
      outfile.write("test accuracy: {:.2f}%\n".format((1 - (1 - model.score(X_test, y_test))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1)))*100))
      outfile.write("roc value : {:.2f}%\n".format(best_roc_value*100))
      outfile.write('\nClassification Report\n\n{}\n\nConfusion Matrix\n\n{}\n'.format(classification_report(y_test, preds),confusion_matrix(y_test, preds)))

  if (file_exists(oldFile("finalmodel","pkl"))==True):
      # Load Model Lama
    old_model = joblib.load(oldFile("finalmodel","pkl"))

    # Cek roc_value model lama
    old_preds = old_model.predict(X_test)
    old_probs = old_model.predict_proba(X_test)[:, 1]

    old_roc_value = roc_auc_score(y_test, old_probs)
    old_train_acc = (1 - (1 - old_model.score(X_train, y_train))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1))) * 100
    old_test_acc = (1 - (1 - old_model.score(X_test, y_test))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))) * 100

    # If statement kalau roc value model lama > model baru = model lama; model baru > model lama = model baru
    if (fin_roc_value > old_roc_value):
      savetotxt(unique_file("finmetrics","txt"), fin_model,fin_preds)
      savetotxt(unique_file("oldmodelmetrics","txt"),old_model, old_preds)
      joblib.dump(fin_model, unique_file("finalmodel", "pkl"))
    else:
      savetotxt("oldmodelmetrics.txt",old_model, old_preds)
      print("Your old model is still better than the new model...")
  
  else:
    savetotxt(unique_file("finmetrics","txt"), fin_model,fin_preds)
    joblib.dump(fin_model, unique_file("finalmodel", "pkl"))

In [ ]:
if __name__ == '__main__':
  fcna = FillCatsNA()
  cd = ConvertData()
  rvt = ReverseVarianceThreshold(threshold = 100)  
  dc = DropCorrelation(0.9)
  main()
  # schedule.every(7).minutes.do(main)
  # while True:
  #   schedule.run_pending()
  #   time.sleep(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DtypeWarning: Columns (42,60,65,82,83,84,85,86) have mixed types.Specify dtype option on import or set low_memory=False.
  


100%|██████████| 10/10 [46:26<00:00, 278.69s/it, best loss: 0.2100546051245492]


In [ ]:
schedule.clear()